# Naive Bayes Classifier

A naive Bayes classifier for detecting spam SMS.  Examples with both a multi-variate Bernoulli event model and a multinomial event model.

Uses SMS Spam Collection v.1 from https://archive.ics.uci.edu/ml/datasets/sms+spam+collection.  A collection of 5572 SMS text messages with 4,827 SMS legitimate messages (86.6%) and a total of 747 (13.4%) spam messages.

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("data\SMSSpamCollection", sep='\t', header=None)
df.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.describe()

,0,1
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


Remove all uppercase SMS messages

In [4]:
df[1] = [row.lower() for row in df[1]]
df.head()

,0,1
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


First we need to split into test and training sets

In [5]:
train_df = df.sample(frac=0.7)
test_df = df.drop(train_df.index)

We now define a function that splits the SMS string into a list of words

In [6]:
def split(s):
    """Split string into a list of words"""
    strings = re.split("[^A-Za-z0-9_&$£'%]+",s)
    #remove empty list items, which can happen at the end using this regex
    strings = [x for x in strings if x]
    return strings

print(split(df[1][2]))
print(split(df[1][3]))

['free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', '21st', 'may', '2005', 'text', 'fa', 'to', '87121', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', "t&c's", 'apply', "08452810075over18's"]
['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']


In [8]:
def train_row(row, dictionary):
    #use a set to easily remove duplicate words (and this Naive Bayes classifier doesn't care about order)
    words = set(split(row))
    for w in words:
        dictionary[w] = dictionary.get(w,0) + 1

Create our training alogrithm.  We want to create a dictionary of ham & spam with count of each word in them.  We also use Laplace smoothing to adjust for words we haven't seen in our training set.

In [7]:
def train(df):
    total_spam = 0
    total_messages = df.shape[0]
    
    ham = {}
    spam = {}
    
    for _, row in df.iterrows():
        if row[0] == "ham":
            train_row(row[1], ham)
        else:
            total_spam += 1
            train_row(row[1], spam)

    total_ham = total_messages - total_spam
    
    post_spam = total_spam/total_messages
    
    #loop through dictionaries to normalise & Laplace smooth
    default_spam = 1/(total_spam + 2)
    default_ham = 1/(total_ham + 2)
    
    for key in iter(ham):
        ham[key] = (ham[key]+1)*default_ham
    for key in iter(spam):
        spam[key] = (spam[key]+1)*default_spam
    
    return spam, ham, post_spam, default_spam, default_ham

Now we train using our training data

In [10]:
spam, ham, post_spam, default_spam, default_ham = train(train_df)

Create our testing alogrithm and test the accuracy of our classifier.  To reduce the rounding error that occurs we've calculated the probablility in a more obscure way.

In [9]:
def test_row(text):
    words = set(split(text))
    p_spam = post_spam
    p_ham = 1-post_spam
    cal = p_ham/p_spam
    #includes Laplace smoothing for words we haven't seen yet by default values
    #Calculates probability in a way that reduces rounding error
    for w in words:
        cal = cal * (ham.get(w, default_ham) / spam.get(w, default_spam))
    prob = 1/(cal + 1)
    return prob

In [11]:
def test(df):
    correct = 0
    total = df.shape[0]
    
    for _, row in df.iterrows():
        if row[0] == "ham" and test_row(row[1]) < 0.5:
            correct += 1
        elif row[0] == "spam" and test_row(row[1]) > 0.5:
            correct += 1
    
    return correct/total

In [12]:
print("Naive Bayes multi-variate")
print("Training accuracy: ", test(train_df))
print("Test accuracy: ", test(test_df))

Naive Bayes multi-variate
Training accuracy:  0.8566666666666667
Test accuracy:  0.7685406698564593


We now build the multinomial event model of the Naive Bayes classifier.  We can reuse a lot of the above code and only need to modify some of the training code and slightly modify some of the testing code

In [13]:
def train_row_MN(row, dictionary):
    """Returns number of words in the row"""
    words = split(row)
    n = len(words)
    for w in words:
        dictionary[w] = dictionary.get(w,0) + 1
    return n

In [14]:
def train_MN(df):
    total_spam = 0
    total_spam_len = 0
    total_ham_len = 0
    total_messages = df.shape[0]
    
    ham = {}
    spam = {}
    
    for _, row in df.iterrows():
        if row[0] == "ham":
            total_ham_len += train_row_MN(row[1], ham)
        else:
            total_spam_len += train_row_MN(row[1], spam)
            total_spam += 1
  
    post_spam = total_spam/total_messages
    
    
    #Get the total unique number of words seen in both ham & spam by combining dictionaries and getting size
    size_dict = len({**ham, **spam})
    default_spam = 1/(total_spam_len + size_dict)
    default_ham = 1/(total_ham_len + size_dict)
    
    #loop through dictionaries to normalise & Laplace smooth
    for key in iter(ham):
        ham[key] = (ham[key]+1)*default_ham
    for key in iter(spam):
        spam[key] = (spam[key]+1)*default_spam
    
    return spam, ham, post_spam, default_spam, default_ham

In [16]:
    spam, ham, post_spam, default_spam, default_ham = train_MN(train_df)

In [17]:
def test_row_MN(text):
    words = split(text)
    p_spam = post_spam
    p_ham = 1-post_spam
    cal = p_ham/p_spam
    
    #includes Laplace smoothing for words we haven't seen yet by default values
    for w in words:
        cal = cal * (ham.get(w, default_ham) / spam.get(w, default_spam))
    
    prob = 1/(1 + cal)
    
    return prob

def test_MN(df):
    correct = 0
    total = df.shape[0]
    
    for _, row in df.iterrows():
        if row[0] == "ham" and test_row_MN(row[1]) < 0.5:
            correct += 1
        elif row[0] == "spam" and test_row_MN(row[1]) > 0.5:
            correct += 1
            
    return correct/total

In [18]:
print("Naive Bayes multinomial:")
print("Training accuracy: ", test_MN(train_df))
print("Test accuracy: ", test_MN(test_df))

Naive Bayes multinomial:
Training accuracy:  0.9951282051282051
Test accuracy:  0.9742822966507177
